In [14]:
import pandas as pd

df =pd.read_csv("/workspaces/Mlflow/predictive_maintenance 1.csv")


In [15]:
df.head()

,ts,ProductID,Type,Airtemperature[°C],Processtemperature[°C],Rotationalspeed[rpm],Torque[Nm],Toolwear[min],FailureType
0,2023-06-01 00:00:00,M14860,M,25.95,36.45,1551,42.8,0,No Failure
1,2023-06-01 00:15:00,L47181,L,26.05,36.55,1408,46.3,3,No Failure
2,2023-06-01 00:30:00,L47182,L,25.95,36.35,1498,49.4,5,No Failure
3,2023-06-01 00:45:00,L47183,L,26.05,36.45,1433,39.5,7,No Failure
4,2023-06-01 01:00:00,L47184,L,26.05,36.55,1408,40.0,9,No Failure


In [16]:
df.dropna(inplace=True)

In [17]:
df['ts'] = pd.to_datetime(df['ts'])  # Convert to datetime if not already
input_x = df['ts']

# Convert datetime to milliseconds since the epoch
df['ts_milliseconds'] = (df['ts'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1ms')

# Now you have the timestamps in milliseconds which you can use for your further processing
input_x = df['ts_milliseconds']
input_x

0       1685577600000
1       1685578500000
2       1685579400000
3       1685580300000
4       1685581200000
            ...      
9995    1694573100000
9996    1694574000000
9997    1694574900000
9998    1694575800000
9999    1694576700000
Name: ts_milliseconds, Length: 10000, dtype: int64

In [19]:
output_x=input_x[:50]

In [18]:
df.head()

,ts,ProductID,Type,Airtemperature[°C],Processtemperature[°C],Rotationalspeed[rpm],Torque[Nm],Toolwear[min],FailureType,ts_milliseconds
0,2023-06-01 00:00:00,M14860,M,25.95,36.45,1551,42.8,0,No Failure,1685577600000
1,2023-06-01 00:15:00,L47181,L,26.05,36.55,1408,46.3,3,No Failure,1685578500000
2,2023-06-01 00:30:00,L47182,L,25.95,36.35,1498,49.4,5,No Failure,1685579400000
3,2023-06-01 00:45:00,L47183,L,26.05,36.45,1433,39.5,7,No Failure,1685580300000
4,2023-06-01 01:00:00,L47184,L,26.05,36.55,1408,40.0,9,No Failure,1685581200000


In [20]:
input_x
input_y=df['Toolwear[min]']

import pandas as pd
from prophet import Prophet

print("Historical Data Range:")
print(f"Start: {pd.to_datetime(min(input_x), unit='ms')}")
print(f"End: {pd.to_datetime(max(input_x), unit='ms')}")

print("\nPrediction Range:")
print(f"Start: {pd.to_datetime(min(output_x), unit='ms')}")
print(f"End: {pd.to_datetime(max(output_x), unit='ms')}")

# Create and prepare the training DataFrame
df = pd.DataFrame()
df['ds'] = pd.to_datetime(input_x, unit='ms')
df['y'] = input_y

# Initialize Prophet model
model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=True
)
model.fit(df)

# Prepare future dates for prediction
future = pd.DataFrame()
future['ds'] = pd.to_datetime(output_x, unit='ms')

# Make predictions
forecast = model.predict(future)
outputY = forecast['yhat'].tolist()

# Debug print for prediction results
print("\nPrediction Summary:")
print(f"Number of predicted points: {len(outputY)}")
print(f"First predicted value: {outputY[0]:.2f}")
print(f"Last predicted value: {outputY[-1]:.2f}")

# return outputY

Historical Data Range:
Start: 2023-06-01 00:00:00
End: 2023-09-13 03:45:00

Prediction Range:
Start: 2023-06-01 00:00:00
End: 2023-06-01 12:15:00


10:36:59 - cmdstanpy - INFO - Chain [1] start processing
10:36:59 - cmdstanpy - INFO - Chain [1] done processing



Prediction Summary:
Number of predicted points: 50
First predicted value: 99.48
Last predicted value: 104.71


In [ ]:
import pandas as pd
import mlflow
import mlflow.prophet
from prophet import Prophet
import logging
import os
from datetime import datetime

# Set MLflow tracking URI to use local filesystem
mlflow.set_tracking_uri("http://localhost:5000")

def setup_mlflow_for_codespaces():
    """
    Configure MLflow for GitHub Codespaces environment
    """
    # Create a directory for MLflow artifacts
    os.makedirs("mlruns", exist_ok=True)
    
    # Set environment variables
    os.environ['MLFLOW_TRACKING_URI'] = "http://localhost:5000"
    
    print("MLflow configured for Codespaces")
    print("To view the MLflow UI:")
    print("1. Run 'mlflow ui --host 0.0.0.0' in a new terminal")
    print("2. Click on the Ports tab in Codespaces")
    print("3. Look for port 5000 and click on the URL or 'Open in Browser' icon")




In [ ]:
def train_and_log_prophet_model(input_x, input_y, output_x, experiment_name="Tool_Wear_Prediction"):
    """
    Train Prophet model and log it with MLflow in Codespaces
    """
    # Set up logging
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)
    
    # Create experiment name with timestamp to avoid conflicts
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    experiment_name = f"{experiment_name}_{timestamp}"
    
    # Set up MLflow experiment
    mlflow.set_experiment(experiment_name)
    
    # Create and prepare the training DataFrame
    df = pd.DataFrame()
    df['ds'] = pd.to_datetime(input_x, unit='ms')
    df['y'] = input_y
    
    # Log data characteristics
    logger.info("Historical Data Range:")
    logger.info(f"Start: {pd.to_datetime(min(input_x), unit='ms')}")
    logger.info(f"End: {pd.to_datetime(max(input_x), unit='ms')}")
    
    with mlflow.start_run() as run:
        # Log parameters
        params = {
            "yearly_seasonality": True,
            "weekly_seasonality": True,
            "daily_seasonality": True
        }
        mlflow.log_params(params)
        
        # Initialize and train Prophet model
        model = Prophet(
            yearly_seasonality=params["yearly_seasonality"],
            weekly_seasonality=params["weekly_seasonality"],
            daily_seasonality=params["daily_seasonality"]
        )
        
        model.fit(df)
        
        # Prepare future dates and make predictions
        future = pd.DataFrame()
        future['ds'] = pd.to_datetime(output_x, unit='ms')
        forecast = model.predict(future)
        outputY = forecast['yhat'].tolist()
        
        # Log metrics
        metrics = {
            "mse": ((forecast['yhat'] - df['y'].mean()) ** 2).mean(),
            "num_predictions": len(outputY)
        }
        mlflow.log_metrics(metrics)
        
        # Log the Prophet model
        mlflow.prophet.log_model(model, "prophet_model")
        
        # Save run info
        run_info = {
            "run_id": run.info.run_id,
            "experiment_id": run.info.experiment_id,
            "artifact_uri": run.info.artifact_uri
        }
        
        # Print prediction summary
        logger.info("\nPrediction Summary:")
        logger.info(f"Number of predicted points: {len(outputY)}")
        logger.info(f"First predicted value: {outputY[0]:.2f}")
        logger.info(f"Last predicted value: {outputY[-1]:.2f}")
        
        # Print MLflow tracking info
        logger.info("\nMLflow Tracking Info:")
        logger.info(f"Run ID: {run_info['run_id']}")
        logger.info(f"Experiment ID: {run_info['experiment_id']}")
        logger.info(f"Artifact URI: {run_info['artifact_uri']}")
        
        return outputY, run_info


In [26]:
def load_and_predict_prophet_model(run_info, output_x):
    """
    Load Prophet model from MLflow and make predictions
    """
    # Load the model from MLflow
    model_uri = f"runs:/{run_info['run_id']}/prophet_model"
    loaded_model = mlflow.prophet.load_model(model_uri)
    
    # Prepare future dates and make predictions
    future = pd.DataFrame()
    future['ds'] = pd.to_datetime(output_x, unit='ms')
    forecast = loaded_model.predict(future)
    outputY = forecast['yhat'].tolist()
    
    print("\nPrediction Results from Loaded Model:")
    print(f"Number of predicted points: {len(outputY)}")
    print(f"First predicted value: {outputY[0]:.2f}")
    print(f"Last predicted value: {outputY[-1]:.2f}")
    
    return outputY



In [27]:
# Example usage
if __name__ == "__main__":
    # Setup MLflow for Codespaces
    setup_mlflow_for_codespaces()
    
    # Train and log the model
    outputY, run_info = train_and_log_prophet_model(input_x, input_y, output_x)
    
    # Load the model and make predictions
    loaded_predictions = load_and_predict_prophet_model(run_info, output_x)

2024/11/13 10:37:58 INFO mlflow.tracking.fluent: Experiment with name 'Tool_Wear_Prediction_20241113_103758' does not exist. Creating a new experiment.
INFO:__main__:Historical Data Range:
INFO:__main__:Start: 2023-06-01 00:00:00
INFO:__main__:End: 2023-09-13 03:45:00
DEBUG:cmdstanpy:input tempfile: /tmp/tmpnvoeyp3p/6x2_ga7p.json


MLflow configured for Codespaces
To view the MLflow UI:
1. Run 'mlflow ui --host 0.0.0.0' in a new terminal
2. Click on the Ports tab in Codespaces
3. Look for port 5000 and click on the URL or 'Open in Browser' icon


DEBUG:cmdstanpy:input tempfile: /tmp/tmpnvoeyp3p/5kdwpshc.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/python/3.12.1/lib/python3.12/site-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=5668', 'data', 'file=/tmp/tmpnvoeyp3p/6x2_ga7p.json', 'init=/tmp/tmpnvoeyp3p/5kdwpshc.json', 'output', 'file=/tmp/tmpnvoeyp3p/prophet_modeloc0e60t_/prophet_model-20241113103759.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
10:37:59 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
10:37:59 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
2024/11/13 10:37:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
INFO:__main__:
Prediction Summary:
INFO:__main__:Number of predicted points: 50
INFO:__main__:Fi


Prediction Results from Loaded Model:
Number of predicted points: 50
First predicted value: 99.48
Last predicted value: 104.71


In [28]:
run_info["run_id"]

'063fec886bbf4e75b281169e612364cf'

In [29]:
import os
import pandas as pd
import mlflow
from mlflow.tracking import MlflowClient
from urllib.parse import urljoin

def load_and_predict_prophet_model(run_id, output_x):
    """
    Load Prophet model from MLflow and make predictions
    
    Parameters:
    -----------
    run_id : str
        The MLflow run ID containing the saved Prophet model
    output_x : list or array-like
        Timestamps in milliseconds for which to make predictions
    
    Returns:
    --------
    list
        Predicted values for the given timestamps
    """
    # Set up MLflow tracking URI
    mlflow_tracking_uri = os.environ.get("MLFLOW_TRACKING_URI", "https://zany-fishstick-xp9vpjwpjp9f6g5j-5000.app.github.dev/")
    mlflow.set_tracking_uri(mlflow_tracking_uri)
    
    # Initialize MLflow client
    client = MlflowClient()
    
    try:
        # Get the run and check if it exists
        run = client.get_run(run_id)
        
        # Construct the model URI using the run ID directly
        model_uri = f"runs:/{run_id}/prophet_model"
        
        # Load the model from MLflow
        loaded_model = mlflow.prophet.load_model(model_uri)
        
        # Prepare future dates and make predictions
        future = pd.DataFrame()
        future['ds'] = pd.to_datetime(output_x, unit='ms')
        
        # Make predictions
        forecast = loaded_model.predict(future)
        outputY = forecast['yhat'].tolist()
        
        print("\nPrediction Results from Loaded Model:")
        print(f"Number of predicted points: {len(outputY)}")
        print(f"First predicted value: {outputY[0]:.2f}")
        print(f"Last predicted value: {outputY[-1]:.2f}")
        
        return outputY
        
    except Exception as e:
        print(f"\nError loading model or making predictions: {str(e)}")
        print(f"MLflow tracking URI: {mlflow_tracking_uri}")
        print(f"Run ID: {run_id}")
        raise


In [31]:
loaded_predictions = load_and_predict_prophet_model(run_info["run_id"], output_x)


Prediction Results from Loaded Model:
Number of predicted points: 50
First predicted value: 99.48
Last predicted value: 104.71
